In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

/usr/local/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/06 07:55:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import numpy as np
import pandas as pd
random_numbers_df = pd.DataFrame(np.random.randint(0,50,size=(5, 2)), columns=list('AB'))

In [4]:
spark_random_numbers_df = spark.createDataFrame(random_numbers_df)

/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [5]:
spark.catalog.listTables()

[]

In [7]:
spark_random_numbers_df.createOrReplaceTempView('temp')

In [10]:
spark.catalog.listTables()

[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [14]:
spark_airports = spark.read.csv('airports.csv', header= True)

In [16]:
type(spark_airports)

pyspark.sql.dataframe.DataFrame

In [19]:
spark_flight_small = spark.read.csv('flights_small.csv', header= True)
spark_flight_small

DataFrame[year: string, month: string, day: string, dep_time: string, dep_delay: string, arr_time: string, arr_delay: string, carrier: string, tailnum: string, flight: string, origin: string, dest: string, air_time: string, distance: string, hour: string, minute: string]

In [20]:
spark_flight_small = spark_flight_small.withColumn('duration_hrs', spark_flight_small['air_time']/60)

In [30]:
flights_further_than_1000 = spark_flight_small.filter('distance > 1000')

In [39]:
flights_further_than_1000.show(1)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|         6.0|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------+
only showing top 1 row



In [43]:
columns_1 = spark_flight_small.select('tailnum', 'origin', 'dest')

In [44]:
columns_1.show(5)

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
+-------+------+----+
only showing top 5 rows



In [69]:
filter1 = columns_1.origin == 'SEA'
filter2 = columns_1.dest == 'PDX'

In [70]:
flights_SEA_to_PDX = columns_1.filter(filter1)
flights_SEA_to_PDX = columns_1.filter(filter2)

In [71]:
spark_flight_small = spark_flight_small.withColumn('avg_speed', spark_flight_small['distance']/spark_flight_small['duration_hrs'])

23/02/06 16:00:19 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 15778010 ms exceeds timeout 120000 ms
23/02/06 16:00:21 WARN SparkContext: Killing executors is not supported by current scheduler.


In [66]:
type(spark_flight_small['avg_speed'])

pyspark.sql.column.Column

In [68]:
# Create a new dataset with each flight's tailnum, origin, destination and full date
cols=["year","month","day"]
flights = flights.withColumn("date",func.concat_ws("-", *cols).cast("date"))
spark_flight_small.selectExpr('tailnum', 'origin', 'dest',"")

  Seq(("2019-01-23",1),("2019-06-24",2),("2019-09-20",3))
    .toDF("date","increment")
    .selectExpr("date","increment","add_months(to_date(date,'yyyy-MM-dd'),cast(increment as int)) as inc_date")
    .show()

AnalysisException: Column 'language' does not exist. Did you mean one of the following? [minute, air_time, arr_time, avg_speed, carrier, day, dep_time, distance, flight, hour, month, origin, tailnum, year, dest, arr_delay, dep_delay, duration_hrs]; line 1 pos 0;
'Project ['language, 'users_count AS count#1720]
+- Project [year#165, month#166, day#167, dep_time#168, dep_delay#169, arr_time#170, arr_delay#171, carrier#172, tailnum#173, flight#174, origin#175, dest#176, air_time#177, distance#178, hour#179, minute#180, duration_hrs#197, (cast(distance#178 as double) / duration_hrs#197) AS avg_speed#1701]
   +- Project [year#165, month#166, day#167, dep_time#168, dep_delay#169, arr_time#170, arr_delay#171, carrier#172, tailnum#173, flight#174, origin#175, dest#176, air_time#177, distance#178, hour#179, minute#180, duration_hrs#197, (cast(distance#178 as double) / duration_hrs#197) AS avg_speed#1287]
      +- Project [year#165, month#166, day#167, dep_time#168, dep_delay#169, arr_time#170, 